In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# datasets

In [ ]:
import os
from PIL import Image as m
from tqdm import tqdm
import random
import cv2
import numpy as np


# mass train dataset paths
mass_images_path = "../input/d/balraj98/massachusetts-buildings-dataset/png/train"
mass_labels_path = "../input/d/balraj98/massachusetts-buildings-dataset/png/train_labels"
mass_image_size = 250

# a train dataset paths
inria_images_path_train = "../input/inria180/training/training/images"
inria_labels_path_train = "../input/inria180/training/training/gt"
inria_image_size = 500
#由一张图像随机生成截取80张500 *500 大小的图像
def createSetsA(image_dir, label_dir, image_size, output_path):
    index = 1
    label_paths = os.listdir(label_dir)
    for path_item in tqdm(label_paths):
       
        #image = m.open(image_dir + "/" + path_item).convert('RGB')
        image = cv2.imread(image_dir + "/" + path_item, cv2.IMREAD_UNCHANGED)
        #label = m.open(label_dir + "/" + path_item.split(".")[0] + ".tif").convert("L")
        label = cv2.imread(label_dir + "/" + path_item, cv2.IMREAD_UNCHANGED)
        X_height, X_width, _ = image.shape
        for key in range(80):
            random_width = random.randint(0, X_width - image_size - 1)
            random_height = random.randint(0, X_height - image_size - 1)
            src_roi = image[random_height: random_height + image_size, random_width: random_width + image_size, :]
            label_roi = label[random_height: random_height + image_size, random_width: random_width + image_size, :]

            cv2.imwrite((output_path + "/images/image%d.tif" % index), src_roi)
            cv2.imwrite((output_path + "/labels/label%d.tif" % index), label_roi)
            index += 1


def createSetsB(image_dir, label_dir, image_size, output_path):
    index = 1
    image_paths = os.listdir(image_dir)
    for path_item in tqdm(image_paths):
        # image = m.open(image_dir + "/" + path_item).convert('RGB')
        image = cv2.imread(image_dir + "/" + path_item, cv2.IMREAD_UNCHANGED)
        # label = m.open(label_dir + "/" + path_item.split(".")[0] + ".tif").convert("L")
        label = cv2.imread(label_dir + "/" + path_item, cv2.IMREAD_UNCHANGED)
        
        src_roi = cv2.resize(image, (image_size,image_size))
        label_roi = cv2.resize(label, (image_size,image_size))
        
        cv2.imwrite((output_path + "/images/image%d.tif" % index), src_roi)
        cv2.imwrite((output_path + "/labels/label%d.tif" % index), label_roi)
        index += 1



def change_label(label_dir):
    image_paths = os.listdir(label_dir)
    for path_item in tqdm(image_paths):
        label = m.open(label_dir + "/" + path_item).convert("L")

        # change 255 to 1
        im_point = label.point(lambda x: x // 255)

        im_point.save(label_dir + "/" + path_item, 'png')

def mkdirs(paths):
    """create empty directories if they don't exist

    Parameters:
        paths (str list) -- a list of directory paths
    """
    if isinstance(paths, list) and not isinstance(paths, str):
        for path in paths:
            mkdir(path)
    else:
        mkdir(paths)


def mkdir(path):
    """create a single empty directory if it didn't exist

    Parameters:
        path (str) -- a single directory path
    """
    if not os.path.exists(path):
        os.makedirs(path)
if __name__ == "__main__":
    # create the paths of created datasets
    mkdirs(['./mass_inria/trainA/images', './mass_inria/trainA/labels','./mass_inria/trainB/images', './mass_inria/trainB/labels'])
    mass_output_path = "./mass_inria/trainA"
    createSetsA(mass_images_path, mass_labels_path, mass_image_size, mass_output_path)

    inria_output_path_train = "./mass_inria/trainB"
    createSetsB(inria_images_path_train, inria_labels_path_train, inria_image_size, inria_output_path_train)

    trainA_label_dir = "./mass_inria/trainA/labels"
    change_label(trainA_label_dir)
    trainB_label_dir = "./mass_inria/trainB/labels"
    change_label(trainB_label_dir)

# Train


## SRS

In [ ]:
import torch.nn as nn
class ResASPPB(nn.Module):
    def __init__(self, channels):
        super(ResASPPB, self).__init__()
        #(in_channels,out_channels,kernel_size,stride,padding,dilation)
        self.conv1_1 = nn.Sequential(nn.Conv2d(channels, channels, 3, 1, 1, 1, bias=False), nn.LeakyReLU(0.1, inplace=True))
        self.conv2_1 = nn.Sequential(nn.Conv2d(channels, channels, 3, 1, 4, 4, bias=False), nn.LeakyReLU(0.1, inplace=True))
        self.conv3_1 = nn.Sequential(nn.Conv2d(channels, channels, 3, 1, 8, 8, bias=False), nn.LeakyReLU(0.1, inplace=True))
        self.conv1_2 = nn.Sequential(nn.Conv2d(channels, channels, 3, 1, 1, 1, bias=False), nn.LeakyReLU(0.1, inplace=True))
        self.conv2_2 = nn.Sequential(nn.Conv2d(channels, channels, 3, 1, 4, 4, bias=False), nn.LeakyReLU(0.1, inplace=True))
        self.conv3_2 = nn.Sequential(nn.Conv2d(channels, channels, 3, 1, 8, 8, bias=False), nn.LeakyReLU(0.1, inplace=True))
        self.conv1_3 = nn.Sequential(nn.Conv2d(channels, channels, 3, 1, 1, 1, bias=False), nn.LeakyReLU(0.1, inplace=True))
        self.conv2_3 = nn.Sequential(nn.Conv2d(channels, channels, 3, 1, 4, 4, bias=False), nn.LeakyReLU(0.1, inplace=True))
        self.conv3_3 = nn.Sequential(nn.Conv2d(channels, channels, 3, 1, 8, 8, bias=False), nn.LeakyReLU(0.1, inplace=True))
        self.b_1 = nn.Conv2d(channels * 3, channels, 1, 1, 0, bias=False)
        self.b_2 = nn.Conv2d(channels * 3, channels, 1, 1, 0, bias=False)
        self.b_3 = nn.Conv2d(channels * 3, channels, 1, 1, 0, bias=False)
    def __call__(self, x):
        buffer_1 = []
        buffer_1.append(self.conv1_1(x))
        buffer_1.append(self.conv2_1(x))
        buffer_1.append(self.conv3_1(x))
        buffer_1 = self.b_1(torch.cat(buffer_1, 1))

        buffer_2 = []
        buffer_2.append(self.conv1_2(buffer_1))
        buffer_2.append(self.conv2_2(buffer_1))
        buffer_2.append(self.conv3_2(buffer_1))
        buffer_2 = self.b_2(torch.cat(buffer_2, 1))

        buffer_3 = []
        buffer_3.append(self.conv1_3(buffer_2))
        buffer_3.append(self.conv2_3(buffer_2))
        buffer_3.append(self.conv3_3(buffer_2))
        buffer_3 = self.b_3(torch.cat(buffer_3, 1))

        return x + buffer_1 + buffer_2 + buffer_3
class ResB(nn.Module):
    def __init__(self, channels):
        super(ResB, self).__init__()
        self.body = nn.Sequential(
            nn.Conv2d(channels, channels, 3, 1, 1, bias=False),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(channels, channels, 3, 1, 1, bias=False),
        )
    def __call__(self,x):
        out = self.body(x)
        return out + x
class SrdanetGenerator(nn.Module):
    def __init__(self, num_cls=2):
        super(SrdanetGenerator, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1, bias=False),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(64, 128, 3, 1, 1, bias=False),
            nn.LeakyReLU(0.1, inplace=True),
            nn.Conv2d(128, 256, 3, 1, 1, bias=False),
            nn.LeakyReLU(0.1, inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            ResB(256),
            ResASPPB(256),
            ResB(256),
            ResASPPB(256),
            ResB(256),
        )

        # downsample == 2
        # SRAAN
        SRAAN_up2 = [nn.ConvTranspose2d(256, 256,
                                         kernel_size=3, stride=2,
                                         padding=1, output_padding=1,
                                         bias=True),
                            nn.ReLU(True)]
        SRAAN_up4 = [nn.ConvTranspose2d(256, 128,
                                         kernel_size=3, stride=2,
                                         padding=1, output_padding=1,
                                         bias=True),
                            nn.ReLU(True)]
        SRAAN_classifier = [nn.Conv2d(128, 3, kernel_size=9, padding=4)]

        self.SRAAN_up2 = nn.Sequential(*SRAAN_up2)
        self.SRAAN_up4 = nn.Sequential(*SRAAN_up4)
        self.SRAAN_classifier = nn.Sequential(*SRAAN_classifier)

        # SSAN
        self.SSAN_mix_conv1 = nn.Conv2d(512, 128, kernel_size=3, padding=1)
        self.SSAN_mix_conv2 = nn.Conv2d(256, 64, kernel_size=3, padding=1)
        self.SSAN_classifier = nn.Conv2d(64, num_cls, kernel_size=9, padding=4)

        # 初始化参数
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    def forward(self, x):
        # feature extractor
        feature = self.feature_extractor(x) # channel=256

        # SRAAN
        SRAAN_up2 = self.SRAAN_up2(feature) # channel=256
        SRAAN_up4 = self.SRAAN_up4(SRAAN_up2) # channel=128
        sr_image = self.SRAAN_classifier(SRAAN_up4) # channel=128

        # SSAN
        _, _, h1, w1 = SRAAN_up2.size()
        SSAN_feature_up2 = nn.functional.interpolate(feature, mode="bilinear", size=(h1, w1), align_corners=True)
        SSAN_feature_up2_cat = torch.cat((SSAN_feature_up2, SRAAN_up2), dim=1) # 512
        SSAN_feature_up2_mix = self.SSAN_mix_conv1(SSAN_feature_up2_cat) # 128

        _, _, h2, w2 = SRAAN_up4.size()
        SSAN_feature_up4 = nn.functional.interpolate(SSAN_feature_up2_mix, mode="bilinear", size=(h2, w2), align_corners=True)
        SSAN_feature_up4_cat = torch.cat((SSAN_feature_up4, SRAAN_up4), dim=1)
        SSAN_feature_up4_mix = self.SSAN_mix_conv2(SSAN_feature_up4_cat)
        pre_seg = self.SSAN_classifier(SSAN_feature_up4_mix)

        return feature, pre_seg, sr_image

## networks

## step1train

In [ ]:
pip install torchsummary

In [ ]:
from torchsummary import summary
import torch 
 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
module = SrdanetGenerator().to(device)
 
summary(module, (3,500 , 500))
